# Εργαστηριακή Άσκηση Υπολογιστικής Νοημοσύνης
## Μερος Β. Αποκατάσταση αρχαίων επιγραφών με χρήση ΓΑ.

Όνομα: Ζαχουλίτης Κωνσταντίνος Γεώργιος <br/>
ΑΜ: 1072578 <br/>
Έτος: 5ο <br/>
Email: up1072578@ac.upatras.gr

### Περιβάλλον Υλοποίησης
CPU:    AMD 6-core 64-bit <br/>
Memory: 16 GB
OS:     Windows 11 23H2 <br/>
Python: 3.12.3 <br/>
IDE:    Pycharm Professional, VS Code

### Β1. Σχεδιασμός ΓΑ

#### α) Κωδικοποίηση

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import re


In [2]:
try:
    df = pd.read_csv("../iphi2802.csv", delimiter="\t", encoding='utf-8')
except:
    df = pd.read_csv("/content/drive/MyDrive/iphi2802.csv", delimiter="\t", encoding='utf-8')
df.head(5)

,id,text,metadata,region_main_id,region_main,region_sub_id,region_sub,date_str,date_min,date_max,date_circa
0,315181,[φ]ιλεταιρος ευμενου περγαμευς μουσαις. καφισι...,Boiotia — Thespiai — mid-3rd c. BC — BCH 26 (1...,1698,Central Greece (IG VII-IX),1691,"Megaris, Oropia, and Boiotia (IG VII)",mid-3rd c. BC,-275.0,-226.0,0.0
1,201686,μαλκοιδων ηρωνος.,"Crete, W. — Tarrha — 1st-3rd c. AD — IC II xxi...",1699,"Aegean Islands, incl. Crete (IG XI-[XIII])",474,Crete,1st-3rd c. AD,1.0,300.0,0.0
2,153178,βασιλικος.,Makedonia (Bottiaia) — Pella — 3rd/2nd c. BC —...,1692,Northern Greece (IG X),1485,Macedonia,3rd/2nd c. BC,-300.0,-101.0,0.0
3,28582,αισκλαπιει μ [ανεθεκε --].,Epidauria — Epidauros — sinistr. — 6th/5th c. BC,1690,Peloponnesos (IG IV-[VI]),1643,"Epidauria (IG IV²,1)",6th/5th c. BC,-600.0,-401.0,0.0
4,333620,[---]ος αν[εθηκε δαματρι].,Italia — Herakleia (Policoro) — late 4th/early...,1696,"Sicily, Italy, and the West (IG XIV)",1689,"Italy, incl. Magna Graecia",late 4th/early 3rd c. BC,-350.0,-251.0,0.0


In [3]:
df = df.drop(['region_main', 'metadata', 'region_sub', 'date_str', 'date_circa'], axis=1)
df=df.dropna()
df.head(5)

,id,text,region_main_id,region_sub_id,date_min,date_max
0,315181,[φ]ιλεταιρος ευμενου περγαμευς μουσαις. καφισι...,1698,1691,-275.0,-226.0
1,201686,μαλκοιδων ηρωνος.,1699,474,1.0,300.0
2,153178,βασιλικος.,1692,1485,-300.0,-101.0
3,28582,αισκλαπιει μ [ανεθεκε --].,1690,1643,-600.0,-401.0
4,333620,[---]ος αν[εθηκε δαματρι].,1696,1689,-350.0,-251.0


In [ ]:
sign_main_ids=df[df['region_main_id'] == 1683]
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(sign_main_ids['text'])
arr = X_tfidf.toarray()
tfidf_df = pd.DataFrame(X_tfidf.toarray())
sign_main_ids['Text_IDF'] = tfidf_df.values.tolist()

vocabulary = vectorizer.vocabulary_
vocab_len = len(vocabulary)
print("Vocabulary:", vocabulary)
print(f'Vocabulary length: {vocab_len}')
sign_main_ids

In [9]:
PHRASE = 'αλεξανδρε ουδις'
for word in PHRASE.split():
    if word not in vocabulary:
        vocabulary[word] = len(vocabulary)
vectorizer = TfidfVectorizer()#(vocabulary=vocabulary)
X_tfidf = vectorizer.fit_transform(sign_main_ids['text'])
arr = X_tfidf.toarray()
vocab_len = len(vocabulary)
tfidf_df = pd.DataFrame(arr, columns=vectorizer.get_feature_names_out())
sign_main_ids['Text_IDF'] = tfidf_df.values.tolist()
vocabulary = vectorizer.vocabulary_
vocab_len = len(vocabulary)

C:\Users\kzach\AppData\Local\Temp\ipykernel_18816\1654143615.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sign_main_ids['Text_IDF'] = tfidf_df.values.tolist()


#### β) Πλεονάζουσες τιμές

In [ ]:
keys_list = list(vocabulary.keys())
unique_keys_set = set(keys_list)

if len(keys_list) != len(unique_keys_set):
    print("There are duplicates in the dictionary.")
else:
    print("No duplicates in the dictionary.")

items_list = list(vocabulary.items())
unique_items_set = set(items_list)

if len(items_list) != len(unique_items_set):
    print("There are duplicate items in the dictionary.")
else:
    print("No duplicates in the items' dictionary.")

#### γ) Αρχικός πληθυσμός

In [34]:
'''Όσον αφορά την κωδικοποίηση των ατόμων του πληθυσμού λάβετε υπόψη τα εξής:
i. κάθε άτομο του πληθυσμού αναπαριστά τις 2 λέξεις που πιθανολογείται ότι λείπουν
από την επιγραφή και υποθέτουμε ότι υπάρχουν στο λεξικό.
ii. κάθε λέξη που υπάρχει στο λεξικό αντιστοιχεί σε έναν ακέραιο στο διάστημα [1,
1678].
iii. Με αυτές τις 2 λέξεις συμπληρώνεται η επιγραφή (δεν έχει σημασία η σειρά),
κατασκευάζεται κατά τα γνωστά το αντίστοιχο διάνυσμα και ελέγχεται η ομοιότητα
του.'''

import random

new_vocab = {k: i for i, k in enumerate(vocabulary)}
for word in PHRASE.split():
    new_vocab[word]=len(new_vocab)
rev_vocab = {index: word for word, index in new_vocab.items()}
print(new_vocab)
print(rev_vocab)


{'οσον': 0, 'επαρκ': 1, 'ει': 2, 'υς': 3, 'και': 4, 'γυ': 5, 'φω': 6, 'αρια': 7, 'χαιρετε': 8, 'αφ': 9, 'ροδιτηι': 10, 'συριηι': 11, 'μητρω': 12, 'αγαθηι': 13, 'τυχηι': 14, 'δημοκρατην': 15, 'αριστογενους': 16, 'προεδρον': 17, 'μεν': 18, 'εισηγη': 19, 'σαμενον': 20, 'ενδοξως': 21, 'δημηγ': 22, 'ορησαντα': 23, 'δις': 24, 'ρξαντα': 25, 'την': 26, 'λαμπρο': 27, 'τατην': 28, 'ρχην': 29, 'πλεονακις': 30, 'πρεσβευ': 31, 'αν': 32, 'τα': 33, 'παρα': 34, 'τους': 35, 'σεβα': 36, 'στους': 37, 'προικα': 38, 'υπερ': 39, 'τω': 40, 'συμφεροντ': 41, 'ων': 42, 'της': 43, 'πατριδος': 44, 'τε': 45, 'ιμηθε': 46, 'ντα': 47, 'υπο': 48, 'πολεως': 49, 'αιδιω': 50, 'αναγ': 51, 'ορευσει': 52, 'θιασαρχησαν': 53, 'πασ': 54, 'λειτουργιαν': 55, 'τελεσαν': 56, 'πολ': 57, 'ειτευσαμενον': 58, 'εν': 59, 'πασιν': 60, 'αγνως': 61, 'τον': 62, 'κτιστην': 63, 'ασυνκ': 64, 'ρι': 65, 'φιλοπατριν': 66, 'κρατιστη': 67, 'βουλη': 68, 'ιερωτατος': 69, 'δημος': 70, 'ευνοιας': 71, 'χαριν': 72, 'ετει': 73, 'υπ': 74, 'ερ': 75, 'κατεσκ

In [ ]:
import random

POPULATION_SIZE = 200
MAX_WORD = 1380

# Λειτουργία για τη δημιουργία αρχικού πληθυσμού
def initialize_population(size):
    population = []
    while len(population) < size:
        individual = [random.randint(1, MAX_WORD), random.randint(1, MAX_WORD)]
        symmetrical_opp = [individual[1],individual[0]]
        if individual not in population and symmetrical_opp not in population:
            population.append(individual)
        else:
            print(f'{symmetrical_opp} and {individual}')
    return population
initial_population = initialize_population(POPULATION_SIZE)

for individual in initial_population[:20]:
    print(individual)


#### δ) Υπολογισμός ομοιότητας

In [36]:
index1 = new_vocab['αλεξανδρε']
index2 = new_vocab['ουδις']
print(index1,index2)

1381 1382


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Υποθέτουμε ότι έχουμε ένα σύνολο επιγραφών σε μορφή λίστας από διανύσματα
# Κάθε επιγραφή αναπαρίσταται ως ένα διάνυσμα συχνοτήτων λέξεων ή TF-IDF
epigrafes = np.array([
    # Παράδειγμα διανυσμάτων επιγραφών (πρέπει να προσαρμοστεί στα δεδομένα σας)
    [0.1, 0.3, 0.5, 0.2],
    [0.0, 0.2, 0.6, 0.1],
    [0.4, 0.1, 0.0, 0.7],
    # Περισσότερες επιγραφές
])

# Υποθέτουμε ότι η επιγραφή που θέλουμε να συγκρίνουμε είναι η πρώτη
target_epigrafi = epigrafes[0].reshape(1, -1)

# Υπολογίζουμε την ομοιότητα συνημιτόνου μεταξύ της δοσμένης επιγραφής και όλων των άλλων
similarities = cosine_similarity(target_epigrafi, epigrafes)

# Εξάγουμε τις top-5 πιο κοντινές επιγραφές
top_k = 5
top_k_indices = np.argsort(similarities[0])[::-1][1:top_k+1]

# Εκτύπωση των top-5 πιο κοντινών επιγραφών
print(f"Top-{top_k} πιο κοντινές επιγραφές:")
for idx in top_k_indices:
    print(f"Επιγραφή {idx + 1} με ομοιότητα {similarities[0][idx]:.4f}")


#### ε) Συνάρτηση καταλληλότητας

In [ ]:
def fitness():
    pass

#### στ) Γενετικοί τελεστές

In [ ]:

# Constants
POPULATION_SIZE = 100
GENERATIONS = 1000
MUTATION_RATE = 0.01
TOURNAMENT_SIZE = 5

# Randomly generate an initial population
def initialize_population(size):
    return [[random.randint(1, 1678), random.randint(1, 1678)] for _ in range(size)]

# Define a dummy fitness function (to be replaced with actual logic)
def fitness_function(individual):
    # This function should measure how well the two words fit the inscription
    # Placeholder: higher indices are considered fitter
    return sum(individual)

# Selection function (tournament selection)
def tournament_selection(population, fitnesses):
    selected = random.sample(list(zip(population, fitnesses)), TOURNAMENT_SIZE)
    selected.sort(key=lambda x: x[1], reverse=True)
    return selected[0][0]

# Crossover function
def crossover(parent1, parent2):
    if random.random() < 0.5:
        return [parent1[0], parent2[1]]
    else:
        return [parent2[0], parent1[1]]

# Mutation function
def mutate(individual):
    if random.random() < MUTATION_RATE:
        individual[random.randint(0, 1)] = random.randint(1, 1678)
    return individual

# Replacement function
def evolve(population):
    fitnesses = [fitness_function(individual) for individual in population]
    new_population = []
    for _ in range(len(population)):
        parent1 = tournament_selection(population, fitnesses)
        parent2 = tournament_selection(population, fitnesses)
        offspring = crossover(parent1, parent2)
        offspring = mutate(offspring)
        new_population.append(offspring)
    return new_population

# Main genetic algorithm loop
population = initialize_population(POPULATION_SIZE)
for generation in range(GENERATIONS):
    population = evolve(population)
    if generation % 100 == 0:
        best_individual = max(population, key=fitness_function)
        print(f"Generation {generation}: Best fitness = {fitness_function(best_individual)}, Best individual = {best_individual}")

# Final result
best_individual = max(population, key=fitness_function)
best_words = [index_to_word[index] for index in best_individual]
print(f"Best words found: {best_words}")


In [ ]:
import pygad
import numpy as np

ga_instance = pygad.GA(num_generations=50,
                       num_parents_mating=4,
                       fitness_func=fitness,
                       sol_per_pop=10,
                       num_genes=5,
                       init_range_low=-4,
                       init_range_high=4,
                       mutation_percent_genes=10)

ga_instance.run()
ga_instance.plot_fitness()

In [ ]:
import numpy as np
import pygad
import random

# Define the fitness function
def fitness_func(solution, solution_idx):
    # Here you can define your custom fitness function
    # For demonstration, we'll use the sum of the values
    return np.sum(solution)

# Define the gene space for each gene (0 or 1)
gene_space = [0, 1]

# Define the number of genes (positions)
num_genes = 1678

# Define the initial positions that must remain fixed
given_positions = [1, 1, 0, 0, 1, 1, 0, 0, 1, 0]  # Example given positions
num_given_positions = len(given_positions)

def on_mutation(ga_instance, offspring_mut):
    # Repair function to ensure no more than two positions are activated
    for solution in offspring_mut:
        if np.sum(solution) > 2:
            indices = [i for i, x in enumerate(solution) if x == 1]
            random.shuffle(indices)
            for idx in indices[2:]:
                solution[idx] = 0
    return offspring_mut

def on_start(ga_instance):
    # Ensure the given positions are set in the initial population
    for solution in ga_instance.population:
        solution[:num_given_positions] = given_positions
    ga_instance.last_generation_fitness = np.apply_along_axis(ga_instance.fitness_func, 1, ga_instance.population)

# Create an instance of the GA class
ga_instance = pygad.GA(
    num_generations=50,
    num_parents_mating=4,
    fitness_func=fitness_func,
    sol_per_pop=100,
    num_genes=num_genes,
    gene_space=gene_space,
    on_start=on_start,
    on_mutation=on_mutation,
    mutation_percent_genes=5
)

# Run the genetic algorithm
ga_instance.run()

# Plot the fitness over generations
ga_instance.plot_fitness()

# Get the best solution
best_solution, best_solution_fitness, _ = ga_instance.best_solution()
print("Best solution:", best_solution)
print("Best solution fitness:", best_solution_fitness)


In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# Δημιουργία του Evaluation Function
def eval_function(individual):
    # Υποθέστε ότι η fitness function είναι η άθροιση των τιμών
    # Αυτή η συνάρτηση μπορεί να προσαρμοστεί ανάλογα με τις ανάγκες σας
    return sum(individual),

# Προετοιμασία του toolbox
toolbox = base.Toolbox()

# Καθορισμός τύπου του ατόμου και του πληθυσμού
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Συνάρτηση για να δημιουργήσουμε ένα άτομο (individual) με 1678 θέσεις
def create_individual():
    individual = [random.randint(0, 1) for _ in range(1678)]
    # Διατήρηση των θέσεων που αντιστοιχούν σε δοσμένες λέξεις σταθερές
    # Εδώ υποθέτουμε ότι οι θέσεις αυτές είναι οι πρώτες 10 θέσεις για το παράδειγμα
    given_positions = [1, 1, 0, 0, 1, 1, 0, 0, 1, 0]  # οι δοσμένες λέξεις
    individual[:10] = given_positions
    return individual

toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Καθορισμός των τελεστών γενετικού αλγόριθμου
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.01)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", eval_function)

# Συνάρτηση επιδιόρθωσης για να διασφαλίσει ότι οι θέσεις δεν ενεργοποιούνται πάνω από 2 φορές
def repair_individual(individual):
    for i in range(len(individual)):
        if individual.count(1) > 2:
            individual[i] = 0
    return individual

toolbox.register("repair", repair_individual)

# Δημιουργία του πληθυσμού
population = toolbox.population(n=100)

# Δημιουργία της συνάρτησης επανάληψης γενετικού αλγόριθμου με επιδιόρθωση
def eaSimpleWithRepair(population, toolbox, cxpb, mutpb, ngen, stats=None,
                       halloffame=None, verbose=__debug__):
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the entire population
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = list(map(toolbox.evaluate, invalid_ind))
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is not None:
        halloffame.update(population)

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cxpb:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < mutpb:
                toolbox.mutate(mutant)
                toolbox.repair(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        if halloffame is not None:
            halloffame.update(offspring)

        # Replace the population with the offspring
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook

# Εκτέλεση του γενετικού αλγόριθμου
final_population, logbook = eaSimpleWithRepair(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, verbose=True)

# Εκτύπωση του καλύτερου αποτελέσματος
best_individual = tools.selBest(final_population, 1)[0]
print(f"Best individual: {best_individual}")
print(f"Fitness: {best_individual.fitness.values[0]}")


### Β2. Υλοποίηση ΓΑ 

### Β3. Αξιολόγηση και Επίδραση Παραμέτρων

#### α) Εκτέλεση του αλγορίθμου για τις τιμές των δοθέντων παραμέτρων.

#### β) Σχεδιασμός της καμπύλης εξέλιξης (απόδοση/αριθμό γενιών) της καλύτερης λύσης 

#### γ) Συμπεράσματά σχετικά με την επίδραση της κάθε παραμέτρου(μέγεθος πληθυσμού, πιθανότητα διασταύρωσης, πιθανότητα μετάλλαξης) στη σύγκλιση του αλγορίθμου